## 교차검증(Cross Validation,CV)

- 모델에 적절한 HyperParameter를 찾기 위해
- 과적합(Overfitting)예방 : 모델이 train 데이터에만 과도하게 최적화되어, 
  실제예측을 다른 데이터로 수행할 경우 예측성능이 과도하게 떨어지는 것

### KFold 교차검증 :  K개의 데이터 폴드 세트 생성 후 K번 만큼 학습과 검증평가를 반복 수행

1) KFold클래스 : 폴드 세트로 분리하는 객체 생성
    - kfold = KFold(n_splits=5)
    
2) split()메소드 : 폴드 데이터 세트로 분리
    - kfold.split(features)
    - 각 폴드마다 학습, 검증용 데이터로 분할할 수 있는 인덱스 반환
    - 학습,검증용 데이터 추출은 인덱스를 기반으로 개발코드에서 직접 수행

In [10]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target

# DecisionTreeClassfier 객체 생성
dt_clf = DecisionTreeClassifier(random_state=156)
# 5개의 폴드 세트로 분리
kfold = KFold(n_splits=5)
# 폴드 세트별 정확도 담을 리스트 생성
cv_accuracy = []

print('붓꽃 데이터 세트 크기 :', features.shape)
n_iter=0


붓꽃 데이터 세트 크기 : (150, 4)
